***This Notebook is used to get a first look at everything before organizing it into .py files and integrating it into a deployable app***

In [3]:
#imports
import pandas as pd
import yfinance as yf


import plotly.express as px
from pathlib import Path

In [4]:
#set the working directory to the folder that the requirments.txt file is in 
def get_root_directory(filename: str = 'requirements.txt'):
    '''Searches the path of the file and parent directories for the requirements.txt file 
    and returns the wd to the folder that contains the requirements.
    This way we can find the absolute path the root
   '''
    current_dir = Path.cwd()
    for parent in [current_dir, *current_dir.parents]: #the * unpacks the parents into a list with multiple entries, otherwise i just use the path object
        if (parent / filename).exists():
            return parent
    #IF we can not find it, raise an error 
    raise FileNotFoundError(f"Could not find {filename} in any parent directory")


In [5]:
get_root_directory()

WindowsPath('c:/Users/steph/Documents/python_scripts/stock_platform')

In [ ]:
ticker_isin = 'US0378331005' #always lengths 12, use appl as example 

ticker = yf.Ticker(f'{ticker_isin}')
ticker.ticker

def isin_ticker_to_ticker(ticker_isin:str):
    '''checks if the input is a ticker or Isin and returns the isin as a ticker for easy use with yfinance'''
    
    if len(ticker_isin) == 12:
        ticker = yf.Ticker(f'{ticker_isin}')
        return ticker.ticker
    else:
        return ticker_isin

ticker = isin_ticker_to_ticker(ticker_isin)
ticker

AttributeError: 'str' object has no attribute 'ticker'

In [7]:
data = yf.download(f'{ticker}', period = 'max')


C:\Users\steph\AppData\Local\Temp\ipykernel_3432\2894773984.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(f'{ticker}', period = 'max')
[*********************100%***********************]  1 of 1 completed


In [70]:
yf.Ticker(ticker).info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [74]:
#Lets visualize the Data using Plotly express

def plot_stock_chart_line(data: pd.DataFrame, ticker):
    '''Plots the chart of a Stock as a Line, needs the input from a yfinane df where only one column remains
    to either display the opening or close prices, default will be the close prices '''
    y_name = f"Price in {yf.Ticker(ticker).info['currency']}"
    comp_name = yf.Ticker(ticker).info['displayName']
    
    fig = px.line(data_frame= data,
            x= data.index,
            y = f'{ticker}',
            labels = {ticker: y_name})
    
    fig.update_layout(title= {"text": f'Chart of the {comp_name} Stock', "font":{"size": 20}, "x": 0.5, "y":0.95, "xanchor": "center", "yanchor": "top"})
    
    fig.show()


In [75]:
plot_stock_chart_line(data= data.Close, ticker= ticker)

In [103]:
#Lets look into common metrics like the Vola of the last 30 days and 1 year

def calculate_volatility(timeframe:tuple, data:pd.DataFrame, ticker: str):
    '''This function calculates the volatility of a Stock for a given start and end date given by a tuple (start, end)
    in the format 'year-month-date', includes start state, omits end date '''
    
    data_adjusted = data[ticker].loc[(data.index >= timeframe[0]) & (data.index < timeframe[1])]
    
    vola_timeframe = data_adjusted.std() #already uses degrees of freedom
    
    return vola_timeframe #we could round but we can also have this as exact number and only round for the displayed output

In [104]:
timeframe = ('2025-08-01', '2025-09-01')
check = calculate_volatility(timeframe= timeframe, data= data.Close, ticker= ticker)
check

np.float64(10.29106605689067)

In [105]:
data.index

DatetimeIndex(['1980-12-12', '1980-12-15', '1980-12-16', '1980-12-17',
               '1980-12-18', '1980-12-19', '1980-12-22', '1980-12-23',
               '1980-12-24', '1980-12-26',
               ...
               '2025-09-03', '2025-09-04', '2025-09-05', '2025-09-08',
               '2025-09-09', '2025-09-10', '2025-09-11', '2025-09-12',
               '2025-09-15', '2025-09-16'],
              dtype='datetime64[ns]', name='Date', length=11281, freq=None)